In [1]:
from acevedo_clss_and_fcns import * 
model                                 = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
feature_data                          = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
feature_names                         = pd.read_csv("./results/data/metabolite_names.csv")

/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Add metabolite concentrations

In [24]:
grafo_nx             = cobra_to_networkx(model)
grafo_nx_onlyConcen  = add_metabolite_concentration_features(grafo_nx, feature_data, feature_names)
to_remove = ['na1_c', 'na1_e', 'na1_m',
 'h2o_c', 'h2o_e', 'h2o_m',
 'h_c', 'h_e', 'h_m',
 'k_c', 'k_e', 'k_m',
 'cl_c', 'cl_e', 'cl_m',
 'pi_c', 'pi_e', 'pi_m',
 'h2o2_c' ,'h2o2_e', 'h2o2_m',
 'o2s_c', 'o2s_e', 'o2s_m']

def get_largest_cc(G):
    
  largest_wcc = max(nx.connected_components(nx.Graph(G)), key=len)


  # Create a subgraph SG based on G
  SG = G.__class__()
  SG.add_nodes_from((n, G.nodes[n]) for n in largest_wcc)


  SG.add_edges_from((n, nbr, d)
      for n, nbrs in G.adj.items() if n in largest_wcc
      for nbr, d in nbrs.items() if nbr in largest_wcc)

  SG.graph.update(G.graph)

  assert G.nodes.__len__() >= SG.nodes.__len__()
  assert G.edges.__len__() >= SG.edges.__len__()
  assert SG.nodes.__len__() == len(largest_wcc)
  #assert not SG.is_directed() 
  assert nx.is_connected(nx.Graph(SG))

  return copy.deepcopy(SG)
grafo_nx_onlyConcen.remove_nodes_from(to_remove)

grafo_nx_onlyConcen = get_largest_cc(grafo_nx_onlyConcen)


pyg_graph_onlyConcen = make_PYG_graph_from_grafo_nx(grafo_nx_onlyConcen)

In [25]:
labels                 = torch.tensor(feature_data.label).reshape(len(feature_data.label),1)
pyg_graph_onlyConcen.y = labels
torch.save(pyg_graph_onlyConcen, "./results/graphs_from_PYG_and_NX/PYG_graph_only_Concen.pt")

## Add fluxes

In [26]:

flux_samples                = pd.read_parquet( "./results/data/flux_samples.parquet.gzip")
grafo_nx_Concen_plus_Fluxes = add_flux_features(grafo_nx_onlyConcen, flux_samples, feature_data)

In [27]:
pyg_graph_Concen_plus_Fluxes = make_PYG_graph_from_grafo_nx(grafo_nx_Concen_plus_Fluxes)

In [28]:
pyg_graph_Concen_plus_Fluxes.y = labels
torch.save(pyg_graph_Concen_plus_Fluxes, "./results/graphs_from_PYG_and_NX/PYG_graph_Concen_plus_Fluxes.pt")

In [29]:
list(grafo_nx_Concen_plus_Fluxes.nodes).index('phe_L_c')


nx.write_gpickle(grafo_nx_Concen_plus_Fluxes, "./results/graphs_from_PYG_and_NX/grafo_nx_Concen_plus_Fluxes.gpickle")